In [1]:
import random 

import numpy as np
import sympy as sp

from spatialmath import SO3, SE3
from roboticstoolbox import ET as et
from spatialmath.base import symbol

from manipylator import Robot
from manipylator.utils import render_robot_from_template, print_collapsible


# Introduction to Spatial Math
The `spatialmath` library provides primitives for manipulating [mathematical objects common in robotics](https://bdaiinstitute.github.io/spatialmath-python/intro.html#), such as `SE3`, `SO3` and `Twist`. Furthermore, this library overloads the `*` python operator to provide for the composition of these objects. Lastly, `spatialmath` is used to implement logic for Elementry Transform & Elementary Transform Sequence from the `robotictoolbox-python` library. For more details see [Robotics Toolbox Python documentation](https://petercorke.github.io/robotics-toolbox-python/intro.html#spatial-math-layer)

As a simple example, here's a 2 degrees-of-freedom planar manipulator, with a Z offset:

In [2]:
transform = et.Rx() * et.tx(1) * et.Ry() * et.tz(1) 
transform

Rx(q0) ⊕ tx(1) ⊕ Ry(q1) ⊕ tz(1)


In [3]:
type(transform)

roboticstoolbox.robot.ETS.ETS

This is a composition of elementary transforms (rotate around x,y,z and translate along xyz), also known as an ETS. This formulation supports symbolic computation directly, and lets us derive symbolic for forward kinematics:

In [4]:
phi, theta, psi = symbol('φ, ϴ, ψ')

type(transform.fkine([phi, theta]))

spatialmath.pose3d.SE3

In [5]:
transform.fkine([phi, theta])

  1.0*cos(ϴ)   0            1.0*sin(ϴ)   1.0*sin(ϴ) + 1.0  
  1.0*sin(φ)*sin(ϴ) 1.0*cos(φ)   -1.0*sin(φ)*cos(ϴ) -1.0*sin(φ)*cos(ϴ)  
  -1.0*sin(ϴ)*cos(φ) 1.0*sin(φ)   1.0*cos(φ)*cos(ϴ) 1.0*cos(φ)*cos(ϴ)  
  0            0            0            1.00000000000000  


Or equivalently, as [sympy.Matrix](https://docs.sympy.org/latest/tutorials/intro-tutorial/matrices.html) objects:

In [6]:
symbolic_forward_kinematic = sp.Matrix(transform.fkine([phi, theta]).A)
symbolic_forward_kinematic

Matrix([
[        1.0*cos(ϴ),          0,         1.0*sin(ϴ),   1.0*sin(ϴ) + 1.0],
[ 1.0*sin(φ)*sin(ϴ), 1.0*cos(φ), -1.0*sin(φ)*cos(ϴ), -1.0*sin(φ)*cos(ϴ)],
[-1.0*sin(ϴ)*cos(φ), 1.0*sin(φ),  1.0*cos(φ)*cos(ϴ),  1.0*cos(φ)*cos(ϴ)],
[                 0,          0,                  0,                1.0]])

Which allows us to leverage sympy for our analysis, for example by calcualting the derivative and jacobian this homogenous transform:

In [7]:
sp.diff(symbolic_forward_kinematic, phi, theta)

Matrix([
[                0, 0,                 0,                 0],
[1.0*cos(φ)*cos(ϴ), 0, 1.0*sin(ϴ)*cos(φ), 1.0*sin(ϴ)*cos(φ)],
[1.0*sin(φ)*cos(ϴ), 0, 1.0*sin(φ)*sin(ϴ), 1.0*sin(φ)*sin(ϴ)],
[                0, 0,                 0,                 0]])

In [8]:
jacobian = symbolic_forward_kinematic[:3, 3].jacobian(sp.Matrix([phi, theta]))
jacobian

Matrix([
[                 0,         1.0*cos(ϴ)],
[-1.0*cos(φ)*cos(ϴ),  1.0*sin(φ)*sin(ϴ)],
[-1.0*sin(φ)*cos(ϴ), -1.0*sin(ϴ)*cos(φ)]])

Or numerically, if we prefer:

In [9]:
values = {phi: 0.1, theta: 0.6}
jacobian.subs(values).evalf(n=3)

Matrix([
[      0,  0.825],
[ -0.821, 0.0564],
[-0.0824, -0.562]])

In [10]:
values = {phi: 0, theta: 0}
jacobian.subs(values).evalf(n=3)

Matrix([
[   0, 1.0],
[-1.0,   0],
[   0,   0]])

# Symbolic Analysis of 6DOF Manipulator
The same powerful symbolic capabilities that spatialmath & SymPy provide are available for a generic, URDF specified robot.

In [11]:
with render_robot_from_template("robots/empiric") as robot_urdf:
    manny = Robot(robot_urdf)
print(manny.model)

ERobot: measured, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │                             ETS: parent to link                             │
├──────┼──────────────────┼───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│    0 │ base             │       │ BASE             │ SE3()                                                                       │
│    1 │ carriage_1       │     0 │ base             │ SE3(-7.47e-06, -2.937e-06, 0.037; -180°, -7.914e-41°, -6.361e-15°) ⊕ Rz(q0) │
│    2 │ shoulder_lift    │     1 │ carriage_1       │ SE3(-0.065, -2.469e-06, -0.055; 180°, 90°, 180°) ⊕ Rz(q1)                   │
│    3 │ elbow            │     2 │ shoulder_lift    │ SE3(0.355, -2.338e-06, -4.032e-06; -180°, -3.45e-15°, 3.069e-12°) ⊕ Rz(q2)  │
│ 

The ETS is available directly, and can be evaluated symbolically:

In [12]:

robot = manny.model

# Get the ETS
ets = robot.ets()

# Create symbolic variables for each joint
q_sym = sp.symbols(f'q0:{robot.n}')

# Substitute symbolic variables into the ETS to get a symbolic SE3
T = ets.eval(q_sym)  # This returns an SE3 object with symbolic entries


As expected, we get a 4x4 matrix, where each element is a symbolic statement. It should be clear here that it's impractical to calculate these kinds of statements by hand.

In [13]:
print("Shape:", T.shape)
print("First element:")
print_collapsible(T[0, 0])

Shape: (4, 4)
First element:


In [14]:
# For the curious what the entire matrix looks like, uncomment below
#print(sp.latex(T))  # LaTeX string

This approach, lets us convert our ETS model to a `sympy.Matrix` which provides a path for further analysis:

In [15]:
T_symbolic = sp.Matrix(T)

Like for example forward kinematics

In [16]:
subs_dict = {q: 0 for q in q_sym}
T_numeric = T_symbolic.subs(subs_dict).evalf(n=3)
T_numeric

Matrix([
[-8.98e-6, -3.67e-6,     1.0, -0.129],
[     1.0, -2.65e-6, 8.98e-6, 1.6e-6],
[ 2.65e-6,      1.0, 3.67e-6,  0.811],
[       0,        0,       0,    1.0]])

Or finding the analytic jacobian:

In [17]:
jacobian = T_symbolic[:3, 3].jacobian(sp.Matrix(q_sym))
print("Shape:", jacobian.shape)

Shape: (3, 6)


In [18]:
print("First element:")
print_collapsible(jacobian[0, 0])

First element:


As should be clear by now, the exact symbolic representation of 6DOF manipulator can be somewhat difficult to write down, but relatively straightforward to wrangle with SymPy. For example, we can expand all terms:

In [19]:
jacobian = jacobian.expand()
print_collapsible(jacobian[0, 0])

Similarly, studying the coefficients of the above expression might be interesting. Specifically, we might be able to find opportunities for simplification:

In [20]:
coeff_dict = jacobian[0, 0].as_coefficients_dict()
print_collapsible(coeff_dict)

The above (likely over 100) coefficients should be understood as a just another way of writing down the expression. For practical reasons, we might even want to reconstruct another expression from it's elements:

In [21]:
threshold = 1e-5 # Chosen arbitrarily to make a point
# Filter coeff_dict
significant_terms = {term: coeff for term, coeff in coeff_dict.items() 
                    if abs(coeff) > threshold} 

# Recreate expression from significant terms
filtered_expression = sum(coeff * term for term, coeff in significant_terms.items())
print("Filtered expression:")
print_collapsible(filtered_expression)

Filtered expression:


In [22]:
filtered_expression.as_coefficients_dict()

defaultdict(int,
            {sin(q0): 0.0885038543428136,
             sin(q0)*cos(q4): 0.0399999999991786,
             sin(q1)*cos(q0): -0.355003000000000,
             sin(q2)*cos(q0)*cos(q1): 0.275498999999030,
             sin(q1)*cos(q0)*cos(q2): -0.275499000000000,
             sin(q2)*cos(q0)*cos(q1)*cos(q3): 0.0885117999993612,
             sin(q3)*cos(q0)*cos(q1)*cos(q2): -0.0885117999990496,
             sin(q1)*cos(q0)*cos(q2)*cos(q3): -0.0885117999996728,
             sin(q1)*sin(q2)*sin(q3)*cos(q0): -0.0885117999993612,
             sin(q4)*cos(q0)*cos(q1)*cos(q2)*cos(q3): 0.0399999999997183,
             sin(q2)*sin(q3)*sin(q4)*cos(q0)*cos(q1): 0.0399999999998592,
             sin(q1)*sin(q2)*sin(q4)*cos(q0)*cos(q3): 0.0399999999998592,
             sin(q1)*sin(q3)*sin(q4)*cos(q0)*cos(q2): -0.0400000000000000})

The above is to say that even when working symbolicly, an order of magnitude analysis might be useful to make a problem tractable. Calculating a symbolic determinant might be impractical on the original expression, but practically trivial in the simplified case.

Due note that the above example is only instructional, and in a practical case use `sympy.nsimplify`:

In [23]:
filtered_expression = sp.nsimplify(jacobian[0, 0], tolerance=1e-5)
filtered_expression.as_coefficients_dict()


defaultdict(int,
            {sin(q0): 2365/26722,
             sin(q1)*cos(q0): -25442/71667,
             sin(q0)*cos(q4): 1/25,
             sin(q1)*cos(q0)*cos(q2): -14051/51002,
             sin(q2)*cos(q0)*cos(q1): 14051/51002,
             sin(q3)*cos(q0)*cos(q1)*cos(q2): -8690/98179,
             sin(q1)*cos(q0)*cos(q2)*cos(q3): -8690/98179,
             sin(q1)*sin(q2)*sin(q3)*cos(q0): -8690/98179,
             sin(q2)*cos(q0)*cos(q1)*cos(q3): 8690/98179,
             sin(q1)*sin(q3)*sin(q4)*cos(q0)*cos(q2): -1/25,
             sin(q4)*cos(q0)*cos(q1)*cos(q2)*cos(q3): 1/25,
             sin(q2)*sin(q3)*sin(q4)*cos(q0)*cos(q1): 1/25,
             sin(q1)*sin(q2)*sin(q4)*cos(q0)*cos(q3): 1/25})

Or for the entire jacobian:

In [24]:
# We want elementwise filtering, therefore use the .applyfunc method
# sympy.nsimplify(sympy.Matrix) does not work as we'd like in this case
filtered_jacobian = jacobian.applyfunc(lambda x: sp.nsimplify(x, tolerance=1e-5))

The simplification can even be quantified by counting the number of terms in the expression:

In [25]:
def count_total_terms(matrix):
    return sum(
        len(element.args) if element.is_Add else 1
        for element in matrix
    )

print(f"Total terms: {count_total_terms(jacobian)}")
print(f"Total filtered terms: {count_total_terms(filtered_jacobian)}")

Total terms: 2433
Total filtered terms: 131


# Calculating Manipulability
By taking the oppurtunity to simplify our Jacobian, further symbolic & numeric calculations can be sped up significantly without loosing much precision. We can now try to calculate a [manipulability metric](https://petercorke.github.io/robotics-toolbox-python/arm_superclass.html#roboticstoolbox.robot.Robot.Robot.manipulability):

In [26]:
# manipulability = sympy.sqrt((jacobian * jacobian.T).det())  # J*J^T is 3×3
# manipulability = sympy.sqrt((filtered_jacobian * filtered_jacobian.T).det()) 


Unfortunately, the above expression does not return in a timely manner, even for the filtered expression. Specifically, the calculating the determinant is the root cause, which could potentially be optimized around by filtering more agressively or waiting longer. Alternatively, a numeric approach might prove more efficient in this case.

Specifically, we can derive a numeric function from our expression and use numpy to efficiently calculate the determinant numerically.

In [27]:
M = filtered_jacobian * filtered_jacobian.T
M_fn = sp.lambdify(q_sym, M, modules='numpy') # Make a wrapper function that handles numpy object

In [28]:
# Create a NxNxN meshgrid for efficiently calculating M_fn
N = 100

q1_vals = np.linspace(-np.pi/2, np.pi/2, N)
q2_vals = np.linspace(-np.pi/2, np.pi/2, N)
q3_vals = np.linspace(-np.pi/2, np.pi/2, N)
Q1, Q2, Q3 = np.meshgrid(q1_vals, q2_vals, q3_vals, indexing='ij')

# Calculate M_fn at meshgrid
# We're simplifying and only looking at DOFs that are likely to have singularities
args = [0, Q1, Q2, Q3, 0, 0]   # [q0, q1, q2, q3, q4, q5]

M_grid = np.asarray(M_fn(*args)) # evaluate; shape is (3,3,N,N,N)
M_grid = np.moveaxis(M_grid, (0,1), (-2,-1))  # -> (N,N,N,3,3) so we can pass to np.linalg.det

In [29]:
# manipulability on the grid: sqrt(det(J*J^T))
det_grid = np.linalg.det(M_grid)
manip_grid = np.sqrt(det_grid)

Once calculated, we can explore the result interactively (if run locally using the lab environment). In this interpretation, low manipulability values indicate a near-singularity configuratation:

In [30]:
import xarray as xr, holoviews as hv, hvplot.xarray, panel as pn
hv.extension('bokeh')

da = xr.DataArray(manip_grid, 
                  coords={'q1': q1_vals, 'q2': q2_vals, 'q3': q3_vals},
                  dims=('q1','q2','q3'), 
                  name='manip')

idx = pn.widgets.IntSlider(name='q3', start=0, end=len(q3_vals)-1, value=len(q3_vals)//2)

@pn.depends(idx)
def view(k):
    return da.isel(q3=k).hvplot.image(x='q1', y='q2', cmap='Viridis', colorbar=True, aspect=1)

pn.Column(idx, "# Yoshikawa Manipulability", view).servable()

Column
    [0] IntSlider(end=99, name='q3', value=50)
    [1] Markdown(str)
    [2] ParamFunction(function, _pane=HoloViews, defer_load=False)

Note the appearance of a low manipulability when q2, q3 are close to zero, indicated by the horizontal darker area. If q3 moves away from 0, this low manipulability area can disappear: 

In [31]:
pn.Column("# Yoshikawa Manipulability for 2 Values of q3" ,pn.Row(view(0), view(50)))

Column
    [0] Markdown(str)
    [1] Row
        [0] HoloViews(Image, sizing_mode='fixed')
        [1] HoloViews(Image, sizing_mode='fixed')

Note: this manipulability measure has units and is scale-dependent. Use it only for relative comparison within the same robot and unit system.

# Conclusion
The above demonstrates how to perform symbolic analysis of a robotic system. We start with a URDF description, convert it into an ETS and then derive a symbolic expression for the pose of the robot. By selectively filtering out small coefficients we distil the expression into a more computationally tractable form. After trying (and failing) to push our analysis further using symbolic means, we switched to numerically calculating the Yoshikawa manipulability metric and identified near-singularity configurations.

# Where To Now?
If you'd like to continue on the symbolic analysis track, check out our [differential kinematics](11-differntial-kinematics.ipynb) notebook. Alternatively, check out how to analyze manipulability using a simulator in our [simulating manipulability](24-simulating-manipulability.ipynb)
